# ETL — Accidents de la route

Ce notebook montre, pas à pas, comment :
1) se connecter à l’API publique Opendatasoft,
2) récupérer un petit échantillon,
3) paginer pour extraire un volume plus grand,
4) sauvegarder les données brutes en CSV,
5) poser les bases du nettoyage (à faire en équipe).

> **Pourquoi ce format ?**  
> Un notebook est idéal pour apprendre : on alterne **explications** (Markdown) et **code** (Python), et on voit les résultats immédiatement.

In [ ]:
# Imports
import pandas as pd
import requests
import time
from pathlib import Path

# Dossiers de sortie
RAW_DIR = Path("../data/raw")
CLEAN_DIR = Path("../data/cleaned")
RAW_DIR.mkdir(parents=True, exist_ok=True)
CLEAN_DIR.mkdir(parents=True, exist_ok=True)


BASE_URL = "https://public.opendatasoft.com/api/explore/v2.1/catalog/datasets/accidents-corporels-de-la-circulation-millesime/records"

SELECT_PARTS = [
    "num_acc",
    "datetime",
    "an",
    "mois",
    "jour",
    "hrmn",
    "lum",
    "agg",
    '"int" as intersection',   # <= réservé → cité + alias
    "atm",
    "col",
    "dep",
    "com",
    "insee",
    "adr",
    "lat",
    '"long" as lon',           # <= réservé → cité + alias
    "surf",
    "circ",
    "nbv",
    "catr",
    "plan",
    "prof",
    "infra",
    "situ",
    "gps",
    "year_georef",
    "dep_name",
    "reg_name",
    "epci_name"
]
SELECT = ", ".join(SELECT_PARTS)


In [ ]:
def fetch_page(offset=0, limit=1000):
    params = {
        "select": SELECT,
        "limit": limit,
        "offset": offset,
        "order_by": "datetime"   # champ 'safe' pour trier
    }
    r = requests.get(BASE_URL, params=params, timeout=60)
    r.raise_for_status()
    return r.json().get("results", [])

# Test rapide
sample = fetch_page(0, 10)
df = pd.DataFrame(sample)
df.head(3)

In [ ]:
TARGET = 5000
PAGE = 1000
DELAY = 0.3

all_rows = []
offset = 0
while offset < TARGET:
    chunk = fetch_page(offset, PAGE)
    if not chunk:
        break
    all_rows.extend(chunk)
    offset += PAGE
    time.sleep(DELAY)

# Sauvegarder un export "brut" pour traçabilité
df_raw = pd.DataFrame(all_rows)
df_raw.to_csv(RAW_DIR / "accidents_sample_raw.csv", index=False)
print(f"{len(df_raw)} lignes enregistrées")


### Chargement des données sources

Jusqu’ici, nous avons vu comment interagir avec l’API publique d’Opendatasoft pour récupérer les données dont nous avons besoin.  
Un fichier d’exemple de 1 000 lignes a permis d’illustrer le principe de pagination et de test de l’API.

Cependant, l’API limite les extractions à des paquets de 100 enregistrements, et le jeu complet (plus de 500 000 lignes) aurait demandé un temps de traitement trop important.  

Pour la suite du brief, nous utiliserons donc directement le fichier CSV complet, déjà téléchargé et préparé à partir de la source officielle :  
👉 [Accidents corporels de la circulation millésimé — Opendatasoft](https://public.opendatasoft.com/explore/assets/accidents-corporels-de-la-circulation-millesime/export/)

Ce fichier servira de base à toutes les étapes suivantes de notre pipeline (nettoyage, transformation et analyse).

### Nettoyage du jeu de données brut
# Ce bloc charge le fichier CSV brut téléchargé depuis Opendatasoft,
# garde uniquement les colonnes utiles à notre modèle, puis écrit un CSV nettoyé dans `data/raw`.

In [ ]:
import pandas as pd


df = pd.read_csv("../data/raw/accidents-corporels-de-la-circulation-millesime.csv",
                     sep=";", dtype=str, encoding="utf-8-sig")


print("Colonnes détectées:", list(df.columns)[:8], "...")

# 2) Garder seulement les colonnes utiles (sans renommage)
cols = [
    "Identifiant de l'accident","Sexe","Date et heure","Commune","Année","Mois","Jour","Heure minute",
    "Lumière","Localisation","Intersection","Conditions atmosphériques","Collision",
    "Département","Code commune","Code Insee","Adresse","Latitude","Longitude",
    "Surface","Circulation","Nombre de voies","Catégorie route","Plan","Profil",
    "Infrastructure","Situation","Gps","year_georef",
    "Nom Officiel Département","Nom Officiel Région","Nom Officiel EPCI","Nom Officiel Commune",
]
df_filtered = df[cols]

out_path = "../data/raw/accidents-corporels-de-la-circulation-millesime_raw.csv"
df_filtered.to_csv(out_path, sep=";", index=False, encoding="utf-8-sig")

print("✅ CSV créé (compatible Excel) :", out_path)


### Nettoyage de base

Nous allons maintenant procéder à un premier nettoyage des données.  
L’objectif ici est d’obtenir un fichier exploitable et cohérent avant d’entamer les transformations plus avancées.

Ce nettoyage de base consiste à :
- supprimer les doublons éventuels,
- éliminer les lignes sans identifiant d’accident,
- préparer un fichier CSV propre dans le dossier `data/cleaned`.

Cette étape garantit que les traitements suivants (analyse, enrichissement, agrégations) reposeront sur un jeu de données unique et fiable.


A PAUFINER

In [ ]:

import pandas as pd
from pathlib import Path

# Charger le fichier déjà filtré
df_raw = pd.read_csv(
    "../data/raw/accidents-corporels-de-la-circulation-millesime_raw.csv",
    sep=";",
    dtype=str,
    encoding="utf-8-sig"
)

print("✅ Fichier chargé :", len(df_raw), "lignes")
print("Colonnes :", list(df_raw.columns)[:8], "...")

# Règles minimales : garder une clé, dédoublonner
df_clean = df_raw.copy()

# supprimer les lignes sans identifiant d'accident
# le nom exact de la colonne est "Identifiant de l'accident"
if "Identifiant de l'accident" in df_clean.columns:
    df_clean = (
        df_clean.dropna(subset=["Identifiant de l'accident"])
                .drop_duplicates(subset=["Identifiant de l'accident"])
    )

print("✅ Après nettoyage :", len(df_clean), "lignes")

# Sauvegarde du jeu nettoyé
CLEAN_DIR = Path("../data/cleaned")
CLEAN_DIR.mkdir(parents=True, exist_ok=True)

clean_csv = CLEAN_DIR / "accidents_clean.csv"
df_clean.to_csv(clean_csv, sep=";", index=False, encoding="utf-8-sig")

print("💾 Fichier nettoyé enregistré :", clean_csv)


In [ ]:
import numpy as np
import pandas as pd

print("🧪 Vérifications basiques")

# 1️⃣ Vérifier les doublons sur la clé
duplicates = df_clean["Identifiant de l'accident"].duplicated().sum()
print(f"🔹 Doublons sur Identifiant de l'accident : {duplicates}")

# 2️⃣ Vérifier les valeurs manquantes globales
missing = df_clean.isna().mean().sort_values(ascending=False)
print("\n🔹 Colonnes avec valeurs manquantes (top 10) :")
print(missing.head(10).round(3))

# 3️⃣ Vérifier la cohérence des dates
if "Date et heure" in df_clean.columns:
    df_clean["Date et heure"] = pd.to_datetime(
        df_clean["Date et heure"], errors="coerce", utc=True
    )
    bad_dates = df_clean["Date et heure"].isna().sum()
    print(f"\n🔹 Dates invalides : {bad_dates}")

# 4️⃣ Vérifier la cohérence géographique
if {"Latitude", "Longitude"} <= set(df_clean.columns):
    df_clean["Latitude"] = pd.to_numeric(df_clean["Latitude"], errors="coerce")
    df_clean["Longitude"] = pd.to_numeric(df_clean["Longitude"], errors="coerce")
    geo_invalid = df_clean[
        (df_clean["Latitude"].isna()) | (df_clean["Longitude"].isna())
    ].shape[0]
    print(f"🔹 Coordonnées invalides ou manquantes : {geo_invalid}")

# 5️⃣ Aperçu des valeurs distinctes (catégorielles)
for col in ["Lumière", "Conditions atmosphériques", "Catégorie route"]:
    if col in df_clean.columns:
        uniques = df_clean[col].dropna().unique()
        print(f"\n🔹 {col} ({len(uniques)} valeurs uniques):")
        print(uniques[:10])

# 6️⃣ Aperçu aléatoire
print("\n🔹 Exemple de lignes :")
display(df_clean.sample(5))

In [ ]:
BRONZE_DIR = Path("../data/bronze")
BRONZE_DIR.mkdir(parents=True, exist_ok=True)
bronze_csv = BRONZE_DIR / "accidents_bronze.csv"

df_clean.to_csv(bronze_csv, sep=";", index=False, encoding="utf-8-sig")
print("💾 Données bronze enregistrées :", bronze_csv)

## Prochaines étapes 

- Générer `dim_time` à partir des dates (ou d’un calendrier)  
- Créer `dim_location` (distinct de `codeinsee`, `departement`, lat/lon si dispo)  
- Créer `dim_conditions` (distinct de `lumiere`, `meteo`, `type_route`, `type_de_collision`)  
- Mapper les IDs (clé naturelle → clé de substitution)  
- Remplir `fact_accident` avec les mesures (`ttue`, `tbg`, `tbl`, `tindm`, `grav`)  
- Écrire `sql/schema.sql` et (option) charger via SQLAlchemy dans SQLite/MySQL/PostgreSQL


In [ ]:
# 🚀 Création et test de connexion PostgreSQL locale

from sqlalchemy import create_engine, text

# Configuration
PG_HOST = "localhost"
PG_PORT = 5432
PG_SU_USER = "postgres"      # superuser par défaut (ou ton user admin)
PG_SU_PASS = "postgres"      # mot de passe du superuser
PG_DB = "accidents"
PG_USER = "accidents"
PG_PASS = "accidents"

# Connexion au superuser (base postgres)
url_su = f"postgresql+psycopg2://{PG_SU_USER}:{PG_SU_PASS}@{PG_HOST}:{PG_PORT}/postgres"
engine_su = create_engine(url_su, future=True)

# Création base + user si absents
with engine_su.begin() as conn:
    conn.execute(text(f"""
    DO $$
    BEGIN
        IF NOT EXISTS (SELECT FROM pg_roles WHERE rolname = '{PG_USER}') THEN
            CREATE ROLE {PG_USER} LOGIN PASSWORD '{PG_PASS}';
        END IF;
        IF NOT EXISTS (SELECT FROM pg_database WHERE datname = '{PG_DB}') THEN
            CREATE DATABASE {PG_DB} OWNER {PG_USER};
        END IF;
    END$$;
    """))
print("✅ Base et utilisateur vérifiés/créés avec succès")

# Connexion à la nouvelle base
url_app = f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}"
engine_app = create_engine(url_app, future=True)

# Test simple : création d’une table temporaire
with engine_app.begin() as conn:
    conn.execute(text("CREATE TABLE IF NOT EXISTS test_conn (id SERIAL PRIMARY KEY, message TEXT);"))
    conn.execute(text("INSERT INTO test_conn (message) VALUES ('Connexion PostgreSQL OK');"))
    msg = conn.execute(text("SELECT message FROM test_conn ORDER BY id DESC LIMIT 1")).scalar_one()

print("🎯 Connexion test réussie —", msg)
## 1) Création (si nécessaire) de la base de données

## 1) Création (si nécessaire) de la base de données

In [ ]:
from pathlib import Path

sql_dir = Path("../sql/bronze").resolve()
sql_files = sorted(sql_dir.glob("*.sql"))
print("Fichiers trouvés:", [p.name for p in sql_files])

db_utils.run_sql_files(sql_files)
print("✅ DDL exécutés (si présents).")

Fichiers trouvés: []
No SQL files provided.
✅ DDL exécutés (si présents).


In [ ]:
created, msg = db_utils.ensure_database_exists()
print("Status:", msg)

Status: ✅ Base 'roadsafety' déjà existante.


In [ ]:
# c:\Users\khagr\Desktop\Formation\projet\etl-road-safety\data\db_utils.py
import os
from sqlalchemy import create_engine, text

def get_cfg():
    return {
        "PG_USER": os.getenv("PG_USER", "postgres"),
        "PG_PASS": os.getenv("PG_PASS", "mot_de_passe_avec_é?@!"),
        "PG_HOST": os.getenv("PG_HOST", "localhost"),
        "PG_PORT": int(os.getenv("PG_PORT", "5432")),
        "PG_DB":   os.getenv("PG_DB", "accident"),
        "PG_SU_USER": os.getenv("PG_SU_USER"),  # optionnel
        "PG_SU_PASS": os.getenv("PG_SU_PASS"),  # optionnel
    }

def _kw(cfg):
    # Connexion directe psycopg2, pas d'URL => pas d'UTF-8 à décoder
    return dict(
        user=cfg["PG_USER"],
        password=cfg["PG_PASS"],
        host=cfg["PG_HOST"],
        port=cfg["PG_PORT"],
        dbname=cfg["PG_DB"],
        options="-c client_encoding=UTF8",
    )

def _kw_super(cfg):
    user = cfg["PG_SU_USER"] or cfg["PG_USER"]
    pwd  = cfg["PG_SU_PASS"] or cfg["PG_PASS"]
    return dict(
        user=user,
        password=pwd,
        host=cfg["PG_HOST"],
        port=cfg["PG_PORT"],
        dbname="postgres",
        options="-c client_encoding=UTF8",
    )

def get_engine():
    import psycopg2
    cfg = get_cfg()
    def _creator():
        return psycopg2.connect(**_kw(cfg))
    # IMPORTANT : on passe un creator → **aucune URL**
    return create_engine("postgresql+psycopg2://", creator=_creator, future=True)

def ensure_database_exists():
    import psycopg2
    cfg = get_cfg()
    dbname = cfg["PG_DB"]

    def _creator_su():
        return psycopg2.connect(**_kw_super(cfg))

    engine_su = create_engine("postgresql+psycopg2://", creator=_creator_su, future=True)

    with engine_su.begin() as conn:
        exists = conn.execute(
            text("SELECT 1 FROM pg_database WHERE datname = :name"),
            {"name": dbname}
        ).fetchone()
        if exists:
            return False, f"✅ Base '{dbname}' déjà existante."
        conn.execute(text(f'CREATE DATABASE "{dbname}"'))
        return True, f"🆕 Base '{dbname}' créée avec succès."


## 1) Création (si nécessaire) de la base de données